## 1. 设$A=\begin{bmatrix} 6 & -4 & 18 \\ 20 &-6 &-6\\22&-22 &11\end{bmatrix}$
## 取 $x^{(0)}=\begin{bmatrix} 1&1&1 \end{bmatrix}^T$
## 求A按模最大的特征值和特征向量，要求误差小于$10^{-6}$
## (1)直接用幂法计算，出现什么现象，说明原因；
## (2)用归一化的幂法计算

### 解：幂法的迭代格式为$x^{k}=Ax^{k-1}$

In [12]:
using LinearAlgebra
A=[6 -4 18;20 -6 -6;22 -22 11] 
x0=[1;1;1]
ϵ=1e-6
iter_num=30

30

In [2]:
#?max

In [3]:
mutable struct MyPowerMethod
    ε::Float64
    max_iter::Int #最大迭代次数   
    iter_idx::Int #迭代次数
    λn::Float64
    xn::Array{Float64,1}
    yn::Array{Float64,1}
    λn_plus::Float64
end

In [4]:
α=x0[argmax(abs.(x0))]*1.0
PM=MyPowerMethod(ϵ,iter_num+20,0,α,copy(x0),x0/α,α)

MyPowerMethod(1.0e-6, 30, 0, 1.0, [1.0, 1.0, 1.0], [1.0, 1.0, 1.0], 1.0)

In [5]:
PM.xn[argmax(abs.(PM.xn))],PM.xn

(1.0, [1.0, 1.0, 1.0])

### 直接迭代，结果如下

In [13]:
for i=1:PM.max_iter
    PM.xn=A*PM.yn
    PM.λn_plus = PM.xn[argmax(abs.(PM.xn))]
    if abs(PM.λn_plus-PM.λn)<PM.ε
        break
    else 
        PM.yn = PM.xn
        PM.λn = PM.λn_plus
        PM.iter_idx+=1
    end
    println(PM.xn)
end
PM

[21.8126, 10.9999, 21.7187]
[477.812, 239.941, 476.785]
[10489.2, 5255.89, 10477.8]
[2.30512e5, 1.15383e5, 2.3039e5]
[5.06856e6, 2.53562e6, 5.06714e6]
[1.11477e8, 5.57546e7, 1.11463e8]
[2.45219e9, 1.22624e9, 2.452e9]
[5.39441e10, 2.69743e10, 5.39427e10]
[1.18674e12, 5.9338e11, 1.18671e12]
[2.61076e13, 1.30542e13, 2.61076e13]


MyPowerMethod(1.0e-6, 10, 20, 2.610762514691538e13, [2.61076e13, 1.30542e13, 2.61076e13], [2.61076e13, 1.30542e13, 2.61076e13], 2.610762514691538e13)

### 可见出现两个摆动的子列$\{x_{2k}\}和\{x_{2k+1}\}$，分析特征值, 发现存在一对相反的特征根，其值如下

In [8]:
eigvals(A)

3-element Array{Float64,1}:
 -22.000000000000007
  21.999999999999993
  10.999999999999993

### 故根据下方的实际计算结果，得到按模最大的特征值 为$\pm 21.99999447621373$

In [7]:
sqrt([4.5331e77, 4.5331e77, 7.25296e77]\[2.19402e80, 2.19402e80, 3.51043e80])

21.99999447621373

### 解：归一化幂法的迭代格式为$x^{k}=Ay^{k-1}$,$\alpha=x^{k}_r, y^{k}=\frac{x^k}{\alpha}$
### 此处我们使用原点移位法，分别用$A-11I$，和$A+11I$迭代，得到特征值以及特征向量。

In [28]:
#负的按模最大特征值特征值和特征向量
PM=MyPowerMethod(ϵ,iter_num,0,α,copy(x0),x0/α,α)
for i=1:PM.max_iter
    PM.xn=(A-11*diagm(0=>[1,1,1]))*PM.yn
    PM.λn_plus = PM.xn[argmax(abs.(PM.xn))]
    if abs(PM.λn_plus-PM.λn)<PM.ε
        break
    else 
        PM.yn = PM.xn/PM.λn_plus
        PM.λn = PM.λn_plus
        PM.iter_idx+=1
    end
    println("iter:",i,", x: ", PM.xn," λ:",11+PM.λn_plus," y: ",PM.yn)
    println("---------------------------------------------------")
end
println("\n特征值:",11+PM.λn_plus,"\n特征向量:",PM.xn)

iter:1, x: [9.0, -3.0, 0.0] λ:20.0 y: [1.0, -0.333333, 0.0]
---------------------------------------------------
iter:2, x: [-3.66667, 25.6667, 29.3333] λ:40.33333333333333 y: [-0.125, 0.875, 1.0]
---------------------------------------------------
iter:3, x: [15.125, -23.375, -22.0] λ:-12.375 y: [-0.647059, 1.0, 0.941176]
---------------------------------------------------
iter:4, x: [16.1765, -35.5882, -36.2353] λ:-25.235294117647058 y: [-0.446429, 0.982143, 1.0]
---------------------------------------------------
iter:5, x: [16.3036, -31.625, -31.4286] λ:-20.624999999999996 y: [-0.515528, 1.0, 0.993789]
---------------------------------------------------
iter:6, x: [16.4658, -33.2733, -33.3416] λ:-22.341614906832298 y: [-0.493852, 0.997951, 1.0]
---------------------------------------------------
iter:7, x: [16.4775, -32.8422, -32.8197] λ:-21.842213114754102 y: [-0.501716, 1.0, 0.999314]
---------------------------------------------------
iter:8, x: [16.4962, -33.0302, -33.0377] λ:-2

In [26]:
#正的按模最大特征值特征值和特征向量
PM=MyPowerMethod(ϵ,100,0,α,copy(x0),x0/α,α)
for i=1:PM.max_iter
    PM.xn=(A+11*diagm(0=>[1,1,1]))*PM.yn
    PM.λn_plus = PM.xn[argmax(abs.(PM.xn))]
    if abs(PM.λn_plus-PM.λn)<PM.ε
        break
    else 
        PM.yn = PM.xn/PM.λn_plus
        PM.λn = PM.λn_plus
        PM.iter_idx+=1
    end
    println("iter:",i,", x: ", PM.xn," λ:",-11+PM.λn_plus," y: ",PM.yn)
    println("---------------------------------------------------")
end
println("\n特征值:",-11+PM.λn_plus,"\n特征向量:",PM.xn)

iter:1, x: [31.0, 19.0, 22.0] λ:20.0 y: [1.0, 0.612903, 0.709677]
---------------------------------------------------
iter:2, x: [27.3226, 18.8065, 24.129] λ:16.322580645161292 y: [1.0, 0.688312, 0.883117]
---------------------------------------------------
iter:3, x: [30.1429, 18.1429, 26.2857] λ:19.142857142857142 y: [1.0, 0.601896, 0.872038]
---------------------------------------------------
iter:4, x: [30.2891, 17.7773, 27.9431] λ:19.28909952606635 y: [1.0, 0.586919, 0.922547]
---------------------------------------------------
iter:5, x: [31.2582, 17.3993, 29.3838] λ:20.25817555938038 y: [1.0, 0.556632, 0.940036]
---------------------------------------------------
iter:6, x: [31.6941, 17.1429, 30.4349] λ:20.694124772864928 y: [1.0, 0.540887, 0.960269]
---------------------------------------------------
iter:7, x: [32.1213, 16.9428, 31.2264] λ:21.121294933781797 y: [1.0, 0.527464, 0.97214]
---------------------------------------------------
iter:8, x: [32.3887, 16.8045, 31.7829] λ